# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt",encoding="utf-8") as f:
    real_estate_sales = f.read()

In [2]:
!pip install langchain
from langchain.text_splitter import CharacterTextSplitter

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [5]:
docs = text_splitter.create_documents([real_estate_sales])

In [6]:
docs[1]

Document(page_content='[客户问题] 这辆车的发动机状况如何？\n[销售回答] 发动机经过全面检查，状况良好，性能稳定。')

In [7]:
len(docs)

61

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [9]:
!pip install -U langchain-community

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [12]:
!pip install tiktoken

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/799.3 kB ? eta -:--:--
     --- ------------------------------------ 71.7/799.3 kB ? eta -:--:--
     ---------- --------------------------- 225.3/799.3 kB 2.8 MB/s eta 0:00:01
     ----------------- -------------------- 358.4/799.3 kB 2.8 MB/s eta 0:00:01
     ----------------- -------------------- 358.4/799.3 kB 2.8 MB/s eta 0:00:01
     ----------------- -------------------- 358.4/799.3 kB 2.8 MB/s eta 0:00:01
     ------------------------- ------------ 532.5/799.3 kB 2.0 MB/s eta 0:00:01
     --------------------------- ---------- 583.7/799.3 kB 2.2 MB/s eta 0:00:01
     ------------------------------ ------- 634.9/799.3 kB 2.0 MB/s eta 0:00:01
     ------------------------------------ - 768.0/799.3 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 799.3/799.3 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/268.5 kB ? eta -:--:--

In [14]:
!pip install faiss-cpu

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     --------------------------------------- 0.0/14.6 MB 262.6 kB/s eta 0:00:56
     --------------------------------------- 0.0/14.6 MB 279.3 kB/s eta 0:00:53
     --------------------------------------- 0.1/14.6 MB 476.3 kB/s eta 0:00:31
     --------------------------------------- 0.1/14.6 MB 476.3 kB/s eta 0:00:31
     --------------------------------------- 0.1/14.6 MB 476.3 kB/s eta 0:00:31
     --------------------------------------- 0.1/14.6 MB 359.9 kB/s eta 0:00:41
     --------------------------------------- 0.2/14.6 MB 437.1 kB/s eta 0:00:34
      -------------------------------------- 0.2/14.6 MB 471.4 kB/s eta 0:00:31
      -------------------------------------- 0.2/14.6 MB 471.4 kB/s eta 0:00:31
      -------------------------------------- 0.2/14.6 MB 450.6 kB/s eta 0:0

In [8]:

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url='https://api.xiaoai.plus/v1'))

D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [28]:
query = "车况怎么样?多少万公里了?"

In [29]:
answer_list = db.similarity_search(query)

In [31]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这辆车的行驶里程是多少？
[销售回答] 这辆车的行驶里程相对较低，车况保持得很好。

\d+\.  
[销售回答] 这辆车的保养记录齐全，定期保养，车况非常好。

[客户问题] 这辆车的油耗怎么样？
[销售回答] 这辆车的油耗非常经济，适合日常通勤和长途驾驶。

[客户问题] 这辆车的年检情况怎么样？
[销售回答] 这辆车的年检记录良好，符合所有安全标准。

[客户问题] 这辆车的发动机状况如何？
[销售回答] 发动机经过全面检查，状况良好，性能稳定。



In [32]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [33]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [37]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002D01C149D90>, search_kwargs={'k': 3})

In [38]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这辆车的行驶里程是多少？
[销售回答] 这辆车的行驶里程相对较低，车况保持得很好。

\d+\.  
[销售回答] 这辆车的保养记录齐全，定期保养，车况非常好。

[客户问题] 这辆车的油耗怎么样？
[销售回答] 这辆车的油耗非常经济，适合日常通勤和长途驾驶。

[客户问题] 这辆车的年检情况怎么样？
[销售回答] 这辆车的年检记录良好，符合所有安全标准。



In [39]:
docs = topK_retriever.get_relevant_documents("你们有没有好一点的车？")

In [41]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这辆车有事故记录吗？
[销售回答] 这辆车没有重大事故记录，车况非常可靠。

[客户问题] 这辆车的年检情况怎么样？
[销售回答] 这辆车的年检记录良好，符合所有安全标准。

[客户问题] 这辆车的售后服务怎么样？
[销售回答] 我们提供全面的售后服务，确保您的用车无忧。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [42]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [43]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`销售回答`

In [44]:
docs = retriever.get_relevant_documents(query)

D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [46]:
docs[0].page_content

IndexError: list index out of range

In [47]:
docs[0].page_content.split("[销售回答] ")

IndexError: list index out of range

In [48]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

IndexError: list index out of range

In [49]:
ans

Document(page_content='[客户问题] 这辆车的发动机状况如何？\n[销售回答] 发动机经过全面检查，状况良好，性能稳定。')

#### 尝试各种问题

In [50]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [52]:
query = "我想要省油的车"

print(sales(query))

[]


D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [53]:
print(sales(query, 0.75))

['这辆车的油耗非常经济，适合日常通勤和长途驾驶。', '这辆车的油耗非常经济，适合各种驾驶需求。']


In [54]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['这辆车没有重大事故记录，车况非常可靠。', '保养费用合理，我们提供专业的保养服务。', '保养费用合理，我们提供专业的保养服务。', '内饰保养得非常好，干净整洁，几乎看不出是二手车。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [41]:
!pip install langchain_openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
     -------------------------- ------------- 30.7/46.1 kB ? eta -:--:--
     -------------------------- ------------- 30.7/46.1 kB ? eta -:--:--
     -------------------------- ------------- 30.7/46.1 kB ? eta -:--:--
     -------------------------- ------------- 30.7/46.1 kB ? eta -:--:--
     -------------------------- ------------- 30.7/46.1 kB ? eta -:--:--
     -------------------------- ------------- 30.7/46.1 kB ? eta -:--:--
     -------------------------------------- 46.1/46.1 kB 114.5 kB/s eta 0:00:00


In [55]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5,base_url = 'https://api.xiaoai.plus/v1')
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [56]:
qa_chain({"query": "你们有200万的劳斯莱斯吗？"})

D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们有200万的劳斯莱斯吗？', 'result': '对不起，我不知道。'}

In [58]:
qa_chain({"query": "发动机吵不吵"})

{'query': '发动机吵不吵', 'result': '发动机声音正常，没有异响，运行平稳。'}

In [59]:
print(sales("机头吵不吵"))

[]


D:\DEV-software\Miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [72]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(base_url = 'https://api.xiaoai.plus/v1') ,allow_dangerous_deserialization = True)

In [73]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.3}))

In [74]:
qa_chain({"query": "我想买大车，你们有么"})

APIConnectionError: Connection error.

In [75]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [76]:
qa_chain({"query": "我想买大车，你们有么"})

APIConnectionError: Connection error.

In [77]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [78]:
result = qa_chain({"query": "我想买大车，你们有么"})

APIConnectionError: Connection error.

In [79]:
result

NameError: name 'result' is not defined